This is a Data Exploration on the asian beauty subreddit.  I will be exploring the posts asking for recommendations for oily/acne prone skin and exploring which products are the most recommended in the comments section. 

Reference material:
http://www.storybench.org/how-to-scrape-reddit-with-python/

https://medium.com/@plog397/webscraping-reddit-python-reddit-api-wrapper-praw-tutorial-for-windows-a9106397d75e

https://praw.readthedocs.io/en/latest/index.html


Import the necessary packages.  I initially had trouble importing praw because the notebook was running a different Python than the one in my command prompt so it could not find the import.  I fixed it by checking sys.executable and sys.path to see where the notebook is looking for imports and did path -m pip install praw.

In [2]:
#! usr/bin/env python3
import sys
import praw
import numpy as np
import pandas as pd 
import string
from datetime import datetime
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sys.executable


'/Users/alice1eung/anaconda3/bin/python'

Set up the Reddit API. I am looking at /r/AsianBeauty 

In [3]:
reddit = praw.Reddit(client_id='14_CHARS_IDENTIFIER', 
                     client_secret='27_CHARS_SECRET_ID', 
                     user_agent='SCRIPT_NAME', 
                     username='REDDIT_USER_NAME', 
                     password='REDDIT_LOGIN_PASSWORD')

asian_beauty=reddit.subreddit('AsianBeauty')

Testing the API

In [3]:
top_asian_beauty = asian_beauty.top(limit=25)

for submission in top_asian_beauty:
    print(submission.title)

[FLUFF] When I walk into Sephora and the sales associate tries to show me their new K-beauty endcap
[Fluff] My mom is 65. She's the reason I wear sunscreen every day!
1st time Bubble Clay mask, laughed so hard at myself that I cried
I’ve battled skincare issues since middle school. Today I woke up with clear skin for the first time in 5 years.
I was confident enough to not wear foundation for the first time to work in 8+ years, thanks AB :)
[FOTD] I finally have skin that I'm proud of.
[News] Hate paying extra on overseas shipping for your favorite products? Well you could be paying even more fees for browsing the net, including r/AsianBeauty without Net Neutrality
I'm supposed to look like Kayako (The Grudge), but I think I look like the Matchmaker from Mulan...
[Guide] I made an Amazon list with some of the most popular items for each step
Inissfree lets customers choose whether they want to be bothered by the staff (x-post from r/mildlyinteresting)
[fluff] When your skin is super so

Getting the data set.  I am trying to filter for posts that are asking for best/fav/recommended products for acne or oily

In [28]:
dict = {"title":[],
       "id":[],
       "url":[],
       "created":[],
       "body":[],
       "comms_num":[]}

for submission in asian_beauty.search('best' and 'acne' and 'oily'):
    if "best" or "favourite" or "favorite" or "recommend" in submission.title.lower:
        dict["title"].append(submission.title)
        dict["id"].append(submission.id)
        dict["url"].append(submission.url)
        dict["created"].append(submission.created)
        dict["body"].append(submission.selftext)
        dict["comms_num"].append(submission.num_comments)
    
data = pd.DataFrame(dict)

#fix the date from UNIX time stamp to a readable date format 
def get_date(created):
    return datetime.fromtimestamp(created)

_timestamp = data["created"].apply(get_date)

data = data.assign(timestamp = _timestamp)

data

,title,id,url,created,body,comms_num,timestamp
0,"[Discussion] AB-ers with oily skin, what are t...",69wwqg,https://www.reddit.com/r/AsianBeauty/comments/...,1.494262e+09,"Do you guys have some life hacks, tips and tri...",135,2017-05-08 12:43:43
1,Best/Worst of Products for Oily Skin! [Discuss...,6lmzyl,https://www.reddit.com/r/AsianBeauty/comments/...,1.499388e+09,Hello! \n\nWe've finally reached the stage in ...,116,2017-07-06 20:43:34
2,[Discussion] Best non-drying yet non-oily suns...,99kj16,https://www.reddit.com/r/AsianBeauty/comments/...,1.535029e+09,,153,2018-08-23 09:01:56
3,[Discussion] What is The Best Moisturizers Lot...,9ea4jx,https://www.reddit.com/r/AsianBeauty/comments/...,1.536493e+09,Hello! Comment product name and the price too ...,64,2018-09-09 07:36:25
4,[Discussion] Is oily skin something to be 'cur...,9efoy7,https://www.reddit.com/r/AsianBeauty/comments/...,1.536550e+09,I've been thinking about this a lot lately. W...,32,2018-09-09 23:25:43
5,[Review] & Farmstay Collagen & Hyaluronic Acid...,9gr77m,https://i.redd.it/gytbkend0xm11.jpg,1.537270e+09,,35,2018-09-18 07:19:06
6,[Discussion] How do you manage thick oily hair?!,988mq3,https://www.reddit.com/r/AsianBeauty/comments/...,1.534591e+09,Some background: I am Chinese with thick wavy ...,46,2018-08-18 07:21:36
7,"Rant, Raves, and Mini Reviews",9h1y16,https://www.reddit.com/r/AsianBeauty/comments/...,1.537359e+09,Have a rant about your routine or beauty produ...,77,2018-09-19 08:06:23
8,HAVE A QUESTION? ASK HERE! Daily Help and Ques...,9g859h,https://www.reddit.com/r/AsianBeauty/comments/...,1.537103e+09,Hello and welcome to the Daily Help Thread! Th...,152,2018-09-16 09:07:01
9,HAVE A QUESTION? ASK HERE! Daily Help and Ques...,98qtcl,https://www.reddit.com/r/AsianBeauty/comments/...,1.534770e+09,Hello and welcome to the Daily Help Thread! Th...,208,2018-08-20 09:07:17


Now I am going to look at the comments of submission id 6remns to test how a sample output can look like.  

In [5]:
submission = reddit.submission(id = '6remns')
for top_level_comment in submission.comments:
    print(top_level_comment.body)

**Cosrx Master Pimple Patch**

**Spot Treatment**

**Best**

**Thoughts** If you have a whitehead which you've lanced the patch will help to seal the wound which helps it heal quicker. I usually use a Blemish cream the next night after using this. It's seriously effective at helping to flatten spots  

Edit: edited to reflect the new information I learnt from deckhandcalypsi below. 
**Benton Snail Bee Essence**

**Essence**

**Best**

This is amazing for calming active acne, reducing redness and inflammation, and is slightly hydrating. I can't rave enough about it. It's healed so much of my scarring/PIH. Love love love it!

Edit: I'm an idiot. I meant essence. Forgive me. 
**LJH Tea Tree Essence**

**Essence**

**Best** 

**Thoughts**. This is a lightweight hydrating essence which sinks quickly into my skin. I love this bottle for helping me with redness and calming down erupting acne. I noticed more acne when I stopped using it. It's one of those products you don't realise the value o

The search function for the API doesn't seem to work the way I want to. I get other posts where the title does not include the words I want. 

I decided to do some more preliminary data exploration to figure out how to text mine the reddit posts/comments to get what I want.  The posts and comments do not have consistent formats and I felt that it would be too inaccurate to have so many different formats.  A future take away could be to look further into text mining and sentiment analysis to have a more accurate way to collect data from a variety of posts using the nltk package. 

Instead, for this data exploraiton I will focus on a specific few posts that asked for the "best of products for ___ type skin" since it gives a standard layout such as the link below.  I will still have to use nltk for sentiment analysis to determine if the comment is saying the product is good or bad.  

https://www.reddit.com/r/AsianBeauty/comments/7j4uvw/discussion_bestworst_of_rasianbeauty_2017/

Future thread to use: 
https://www.reddit.com/r/AsianBeauty/comments/756o15/routine_megathread_october_2017/


In [29]:


dict = {"id" : [],
        "product" : [], 
        "step" : [],
        "opinion" : [],
        "thoughts" : [],
        "thread": []
}

submission = reddit.submission(id = '6lmzyl')

#This isn't the most advanced way to text mine unstructured data but I'm gonna leave it
# as this for now cause it mostly works.  Future plans can be to improve this!
for comment in submission.comments:
    if(len(comment.body.split("\n\n")) >= 4 and comment.body[0] == "*"):
        dict["id"].append(comment.id)
        dict["product"].append(comment.body.split("\n\n")[0].replace("**",""))
        dict["step"].append(comment.body.split("\n\n")[1].replace("**",""))
        dict["opinion"].append(comment.body.split("\n\n")[2].replace("**",""))
        dict["thoughts"].append(comment.body.split("\n\n")[3].replace("**",""))  
        dict["thread"].append("Oily")
       
submission = reddit.submission(id = '6na9hw')
for comment in submission.comments:
    if(len(comment.body.split("\n\n")) >= 4):
        dict["id"].append(comment.id)
        dict["product"].append(comment.body.split("\n\n")[0].replace("**",""))
        dict["step"].append(comment.body.split("\n\n")[1].replace("**",""))
        dict["opinion"].append(comment.body.split("\n\n")[2].replace("**",""))
        dict["thoughts"].append(comment.body.split("\n\n")[3].replace("**",""))  
        dict["thread"].append("Dry")
        
submission = reddit.submission(id = '6uijtv')
for comment in submission.comments:
    if(len(comment.body.split("\n\n")) >= 4):
        dict["id"].append(comment.id)
        dict["product"].append(comment.body.split("\n\n")[0].replace("**",""))
        dict["step"].append(comment.body.split("\n\n")[1].replace("**",""))
        dict["opinion"].append(comment.body.split("\n\n")[2].replace("**",""))
        dict["thoughts"].append(comment.body.split("\n\n")[3].replace("**",""))  
        dict["thread"].append("Combination")
        
submission = reddit.submission(id = '6remns')
for comment in submission.comments:
    if(len(comment.body.split("\n\n")) >= 4):
        dict["id"].append(comment.id)
        dict["product"].append(comment.body.split("\n\n")[0].replace("**",""))
        dict["step"].append(comment.body.split("\n\n")[1].replace("**",""))
        dict["opinion"].append(comment.body.split("\n\n")[2].replace("**",""))
        dict["thoughts"].append(comment.body.split("\n\n")[3].replace("**",""))  
        dict["thread"].append("Acne")
        
data = pd.DataFrame(dict)
data

,id,product,step,opinion,thoughts,thread
0,djv0ax0,Innisfree Super Volcanic Clay Mask,Clay Mask,Best,Thoughts I use a clay mask at least once a wee...,Oily
1,djv702q,Belif Aqua Bomb,Moisturizer,Best,I love this stuff. It's a gel cream that sinks...,Oily
2,djv01bn,Blotting Papers,Tools,Best,"Thoughts Not really a product as such, but I s...",Oily
3,djv4eeq,Missha Super Aqua Ultra Water-Full Clear Cream,Moisturizer,Best,Thoughts Using this gel-type moisturizer at ni...,Oily
4,djuzzcn,Innisfree No Sebum Powder - Loose,Make Up,Best,Thoughts Love this affordable powder! I dust i...,Oily
5,djv05rw,Elizavecca 24K Gold Water Drop Cream Mask,Moisturiser,Best,Thoughts This is a lightweight gel from Elizav...,Oily
6,djvlvej,Pyungkang Yul Mist Toner,Hydrating toner,Best,Thoughts: This product is how I'm surviving su...,Oily
7,djvedyg,Cosrx Aloe Vera Oil-free Moisture Cream,Moisturizer,Best,"Thoughts: I've been using this cream, which is...",Oily
8,djvpgxn,Zymogen Houttyunia Cordata Serum and Lotion,Toner and moisturiser,Best,I've pretty much used at least 4 sets of these...,Oily
9,djv8nyi,Neogen Black Volume Cream,Sleeping Pack/Moisturizer,Best,I had purchased this in a shady skinfood brick...,Oily


Experiemnting on how to get a comment forest (the replies to a comment).  Since there are only 54 items in the list and I want to be able to quantify the popularity of a product by applying sentiment analysis on the comments to see if it's a positive comment or negative. 

Also this data exploraiton is becoming very unorganized :( I might make this into the preliminary exploration and then have a more organized final data exploration. 

In [5]:
sid = SentimentIntensityAnalyzer()
for comment in submission.comments:
    comment.refresh()
    for reply in comment.replies:
        print(reply.body)
        ss = sid.polarity_scores(reply.body)
        for k in ss:
            print( "{0}: {1}," .format(k, ss[k]), end=" ")
            print()

Just because I know some people have questions, here is some helpful info on hydrocolloid patches. (Copy+paste from another comment of mine)

I've noticed some misunderstanding around various skin care subs about what, exactly, hydrocolloid bandages do. They don't actually suck the gunk out of pimples!  Initially, they were made for blisters before being repurposed for acne. They absorb moisture from the wound to form a gel, which is why the bandage rapidly turns into a soft and spongy mass which also cushions your wound. The gel on the bandage is also cohesive, which means it creates a seal and whatever leaks from your wound stays in place under the dressing, allowing the wound to heal quicker. So the white stuff on the bandage isn't pus or anything, it is the moisture of the wound interacting with hydrocolloid on the bandage, causing the white appearance. This also makes sense with why they suggest the wound to already by open in some form as the bandage needs to interact with fluid 

I use this too, mixed with a couple drops of TTO and it can zap a pretty nasty breakout. I will often do an exfoliation step before if it is for general maintenance versus single breakout. I have been using this mask since before my AB days, almost 5+ years now. I have no idea how I found it. I follow with foaming cleanser to get the gunk out, than the rest of my routine per normal, often with a sheet mask to get hydration back into my skin. 

I prefer this mask because it is really affordable and will never dry out, since it comes as a powder.

I also have used just the powder on my face when I am really red to set makeup (if I am even wearing any) or just as a color correcting and mattifying powder. *I don't have perfect skin, just lazy AF so not big into makeup although moving to an AB style skin routine over the last 8ish months has cleared my skin considerably.
neg: 0.067, 
neu: 0.905, 
pos: 0.027, 
compound: -0.7716, 
I've had the same results. It helps heal existing pimples but 

So the sentiment analyzer seems to work quite well! I'm going to add a positive count and negative count column to the data set to record the comments. 

In [30]:
sid = SentimentIntensityAnalyzer()
data["pos_count"] = 0
data["neg_count"] = 0
data["neu_count"] = 0
count_row = len(data)

for i in range(0,count_row): 
    comment = reddit.comment(data['id'].iloc[i])
    comment.refresh()
    #print(comment.body)
    for reply in comment.replies:
        #print(reply.body)
        ss = sid.polarity_scores(reply.body)
        
        if ss["compound"] == 0.0:
            #print("neutral")
            data.loc[i, "neu_count" ] += 1
        elif ss["compound"] > 0.0:
            #print("postive")
            data.loc[i, "pos_count" ] += 1
        else:
            #print("neg")
            data.loc[i, "neg_count" ] += 1
        

In [31]:
data.head()

,id,product,step,opinion,thoughts,thread,pos_count,neg_count,neu_count
0,djv0ax0,Innisfree Super Volcanic Clay Mask,Clay Mask,Best,Thoughts I use a clay mask at least once a wee...,Oily,7,2,0
1,djv702q,Belif Aqua Bomb,Moisturizer,Best,I love this stuff. It's a gel cream that sinks...,Oily,3,1,0
2,djv01bn,Blotting Papers,Tools,Best,"Thoughts Not really a product as such, but I s...",Oily,6,0,0
3,djv4eeq,Missha Super Aqua Ultra Water-Full Clear Cream,Moisturizer,Best,Thoughts Using this gel-type moisturizer at ni...,Oily,2,0,0
4,djuzzcn,Innisfree No Sebum Powder - Loose,Make Up,Best,Thoughts Love this affordable powder! I dust i...,Oily,7,2,5


In [32]:
data = data.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)

data.loc[data['opinion'].str.contains("best"), 'pos_count'] = data['pos_count'] + 1
data.loc[data['opinion'].str.contains("worst"), 'pos_count'] = data['pos_count'] + 1


data

,id,product,step,opinion,thoughts,thread,pos_count,neg_count,neu_count
0,djv0ax0,innisfree super volcanic clay mask,clay mask,best,thoughts i use a clay mask at least once a wee...,oily,8,2,0
1,djv702q,belif aqua bomb,moisturizer,best,i love this stuff. it's a gel cream that sinks...,oily,4,1,0
2,djv01bn,blotting papers,tools,best,"thoughts not really a product as such, but i s...",oily,7,0,0
3,djv4eeq,missha super aqua ultra water-full clear cream,moisturizer,best,thoughts using this gel-type moisturizer at ni...,oily,3,0,0
4,djuzzcn,innisfree no sebum powder - loose,make up,best,thoughts love this affordable powder! i dust i...,oily,8,2,5
5,djv05rw,elizavecca 24k gold water drop cream mask,moisturiser,best,thoughts this is a lightweight gel from elizav...,oily,1,0,0
6,djvlvej,pyungkang yul mist toner,hydrating toner,best,thoughts: this product is how i'm surviving su...,oily,3,1,0
7,djvedyg,cosrx aloe vera oil-free moisture cream,moisturizer,best,"thoughts: i've been using this cream, which is...",oily,2,0,0
8,djvpgxn,zymogen houttyunia cordata serum and lotion,toner and moisturiser,best,i've pretty much used at least 4 sets of these...,oily,1,0,0
9,djv8nyi,neogen black volume cream,sleeping pack/moisturizer,best,i had purchased this in a shady skinfood brick...,oily,1,0,0


In [33]:
data.to_csv("skincare.csv", encoding='utf-8', index=False)